In [20]:
import struct
import mido
import msgpack
# clamp bytes to < 127 for sysex by splitting them 
# encode a byte into 2, high 7 bits in first byte, lsb as second
# crazy inefficient, but effective 
def doubled_encode_data(data):
    encoded_data = []
    for byte in data:
        scaled_byte1 = byte >> 1 # everything before lsb
        scaled_byte2 = byte - (scaled_byte1 << 1) # lsb
        # Add scaled bytes to encoded data
        encoded_data.extend([scaled_byte1, scaled_byte2])
    return bytearray(encoded_data)

# MIDI Spec
start_sysex = bytearray([0xF0])
end_sysex = bytearray([0xF7])

# Make message formatted consistently 
def make_tlv_sysex(typeID, data):
    # Only data not encoded/clamped as 2 bytes per byte for sysex as value is stable-ish  
    typeID = struct.pack("h", typeID)
    
    packed_message = msgpack.packb(data) # msgpack so it is "smaller"
    # clamp bytes values to 0-127
    message = bytearray(doubled_encode_data(packed_message)) 

    # Length
    length = doubled_encode_data(struct.pack("h", len(message)))

    # build message 
    sysex_message = bytearray()
    sysex_message.extend(start_sysex) # 0xF0
    sysex_message.extend(typeID)   # T
    sysex_message.extend(length)   # L
    sysex_message.extend(message)  # V
    sysex_message.extend(end_sysex) # 0xF0
    # mido will throw an error on bytes over 127
    midi_sysex_msg = mido.Message.from_bytes(sysex_message)
    return midi_sysex_msg

test_simple_midi = mido.Message('note_on', note=60)

bee_movie = """
# # Bee Movie
# # By Jerry Seinfeld

# # NARRATOR:
# # (Black screen with text; The sound of buzzing bees can be heard)
# # According to all known laws
# # of aviation,
# #  :
# # there is no way a bee
# # should be able to fly.
# #  :
# # Its wings are too small to get
# # its fat little body off the ground.
# #  :
# # The bee, of course, flies anyway
# #  :
# # because bees don't care
# # what humans think is impossible."""
test_msgpack_bee = {
    "bee": bee_movie
}

test_msgpack_arbitrary = {
    "boolean": True,
    "arbitraryArr": [1, False, [], {}],
    "nested": {
        "data": None
    }
}

test_sysex_arbitrary = make_tlv_sysex(0x01, test_msgpack_arbitrary)
test_sysex_bee = make_tlv_sysex(0x01, test_msgpack_bee)

with mido.open_output('MIDI function 1') as outport:
    outport.send(test_simple_midi)
    outport.send(test_sysex_bee)
    outport.send(test_sysex_arbitrary)
   